In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import streamlit as st
import altair as alt

In [2]:
def get_projections():
    live = pd.read_csv(r"https://feeds.datagolf.com/preds/live-tournament-stats?stats=sg_putt,sg_arg,sg_app,sg_ott,sg_t2g,sg_bs,sg_total,distance,accuracy,gir,prox_fw,prox_rgh,scrambling&round=event_avg&display=value&file_format=csv&key=e297e933c3ad47d71ec1626c299e")#,usecols=['dk_name','total_points'])
    return live
live = get_projections()
dg_proj_copy = live.copy()
live.rename(columns={'player_name':'player'},inplace=True)

In [3]:
# fix names 'live' vs 'fx'

names = live['player'].str.split(expand=True)
names[0] = names[0].str.rstrip(",")
names[1] = names[1].str.rstrip(",")
names['player'] = names[1] + " " + names[0]
names['player'] = np.where(names['player']=='Min Lee', 'Min Woo Lee', names['player'])
names['player'] = np.where(names['player']=='Byeong An', 'Byeong Hun An', names['player'])
names['player'] = np.where(names['player']=='Rooyen Van', 'Erik Van Rooyen', names['player'])
live = live.set_index(names.player)

In [4]:
def get_fantrax():
    teams = pd.read_csv(r"fx_wk9.csv",usecols=['Player','Status','Roster Status'])
    return teams
teams = get_fantrax()

In [5]:
teams.columns = ['player','team','active_reserve']
teams_dict = {'919':'Philly919','u_c':'unit_circle','NT 4':'New Team 4','NT 8':'Sneads Foot','txms':'txmoonshine','MG':'Team Gamble','grrr':'Putt Pirates','[AW]':'AlphaWired'}
teams['team'] = teams.team.map(teams_dict)
teams = teams.loc[teams.active_reserve=='Active'].set_index('player')

In [7]:
live_merged = pd.merge(teams, live, how='left', left_index=True, right_index=True)[['player','team','position','total','round','thru','sg_putt','sg_t2g','sg_total','gir']].fillna(0).sort_values('total')
live_merged['holes_remaining'] = (72 - (live_merged['thru']).fillna(0))
live_merged['holes_remaining'] = np.where(live_merged['position']=='CUT',0,live_merged['holes_remaining']).astype('int')
live_merged = live_merged.reset_index(drop=True)

In [9]:
live_merged.head()

,player,team,position,total,round,thru,sg_putt,sg_t2g,sg_total,gir,holes_remaining
0,"Lee, Min Woo",unit_circle,T2,-14.0,-4.0,18.0,0.2450,2.1700,2.415,0.638890,54
1,"Van Rooyen, Erik",New Team 4,T2,-14.0,-8.0,18.0,1.9550,0.4625,2.415,0.750003,54
2,"Knapp, Jake",unit_circle,T4,-13.0,-5.0,18.0,0.3675,1.8000,2.165,0.708333,54
3,"Lowry, Shane",unit_circle,T4,-13.0,0.0,18.0,0.8275,1.3375,2.165,0.805555,54
4,"Young, Cameron",New Team 4,T4,-13.0,-5.0,18.0,0.9750,1.1900,2.165,0.791665,54


In [12]:
team_color={
                "Philly919": 'rgb(127,60,141)',
                "unit_cirle": 'rgb(17,165,121)',
                "AlphaWired": 'rgb(57,105,172)',
                "Sneads Foot": 'rgb(242,183,1)',
                "New Team 4": 'rgb(231,63,116)',
                "Team Gamble": 'rgb(230,131,16)',
                "txmoonshine": 'rgb(0,134,139)',
                "Putt Pirates": 'rgb(165,170,153)'}

In [11]:
def highlight_rows(row):
    value = row.loc['Team']
    if value == 'unit_circle':
        color = '#FF99FF' # Pink
    elif value == 'Philly919':
        color = '#7f3c8d' # Purple
    elif value == 'AlphaWired':
        color = '#3969ac' # Blue
    elif value == 'Sneads Foot':
        color = '#f2b701' # Gold
    elif value == 'New Team 4':
        color = '#FF6666' # Magenta
    elif value == 'Team Gamble':
        color = '#e68310' # Orange
    elif value == 'txmoonshine':
        color = '#00868b' # Aqua
    else:
        color = '#a5aa99' # Grey
    return ['background-color: {}'.format(color) for r in row]



In [13]:
# make leaderboard

live_leaderboard = live_merged[['player','team','position','total','round','thru']].fillna(0).sort_values('total')

live_leaderboard[['total','round','thru']] = live_leaderboard[['total','round','thru']].astype('int')

live_leaderboard.rename(columns={'player':'Player','team':'Team','position':'Pos','total':'Total','round':'Round','thru':'Thru'}).style.apply(highlight_rows, axis=1)

,Player,Team,Pos,Total,Round,Thru
0,"Lee, Min Woo",unit_circle,T2,-14,-4,18
1,"Van Rooyen, Erik",New Team 4,T2,-14,-8,18
2,"Knapp, Jake",unit_circle,T4,-13,-5,18
3,"Lowry, Shane",unit_circle,T4,-13,0,18
4,"Young, Cameron",New Team 4,T4,-13,-5,18
5,"Ryder, Sam",Sneads Foot,T21,-10,-5,18
6,"McIlroy, Rory",Sneads Foot,T21,-10,-3,18
7,"An, Byeong Hun",Team Gamble,T21,-10,-6,18
8,"Hoge, Tom",Team Gamble,T28,-9,-3,18
9,"Pavon, Matthieu",Team Gamble,T28,-9,-1,18


In [15]:
# holes remaining
live_phr = live_merged.groupby('team')[['holes_remaining']].sum().reset_index().rename(columns={'team':'Team','holes_remaining':'PHR','total':'To Par'})
live_phr.sort_values(by='PHR').style.apply(highlight_rows, axis=1)

,Team,PHR
1,New Team 4,162
2,Philly919,162
5,Team Gamble,216
4,Sneads Foot,270
3,Putt Pirates,360
7,unit_circle,360
6,txmoonshine,396
0,AlphaWired,432


In [31]:
# Team Score bar

thru_cut_df = live_leaderboard.groupby('team')['total'].sum().sort_values()

thru_cut_bar = px.bar(thru_cut_df,
                     template='presentation',
                     labels={'value':'','team':''},
                     text_auto=True,
                     height=250,
                     title='Team Score to Par')

thru_cut_bar.update_layout(showlegend=False,title_x=.25)
thru_cut_bar.update_yaxes(showticklabels=False,showgrid=False)
thru_cut_bar.update_traces(marker_color='rgb(200,200,200)',marker_line_width=1.5, opacity=0.6)

thru_cut_bar.show()

In [17]:
# Thru Cut Bar

thru_cut_df = live_leaderboard[live_leaderboard.position!='CUT']['team'].value_counts()#.rename(columns={'team':'thruCut'})

thru_cut_bar = px.bar(thru_cut_df,
                 template='presentation',
                 labels={'value':'','index':''},
                 text_auto=True,
                 height=250,
                 log_y=True,
                 title='Players Thru the Cut')

thru_cut_bar.update_layout(showlegend=False,title_x=.25)
thru_cut_bar.update_yaxes(showticklabels=False,showgrid=False)
thru_cut_bar.update_traces(marker_color='rgb(200,200,200)',marker_line_width=1.5, opacity=0.6)

In [32]:
# live sg
live_sg = live_merged[['player','sg_putt','sg_t2g','sg_total','gir']].reset_index(drop=True)
live_sg = live_sg.style.background_gradient(cmap='Greens').format(precision=2)
live_sg

,player,sg_putt,sg_t2g,sg_total,gir
0,"Lee, Min Woo",0.24,2.17,2.42,0.64
1,"Van Rooyen, Erik",1.96,0.46,2.42,0.75
2,"Knapp, Jake",0.37,1.80,2.17,0.71
3,"Lowry, Shane",0.83,1.34,2.17,0.81
4,"Young, Cameron",0.97,1.19,2.17,0.79
5,"Ryder, Sam",1.16,0.25,1.42,0.74
6,"McIlroy, Rory",-0.90,2.31,1.41,0.78
7,"An, Byeong Hun",0.17,1.24,1.42,0.71
8,"Hoge, Tom",0.30,0.86,1.17,0.74
9,"Pavon, Matthieu",0.83,0.34,1.17,0.71


In [121]:
print(f"unit_circle has {live_merged[live_merged.team=='unit_circle']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='unit_circle'].drop(columns=['holes_remaining','team'])

unit_circle has 360 Holes Remaining


,player,position,total,round,thru,sg_putt,sg_t2g,sg_total,gir
0,"Lee, Min Woo",T2,-14.0,-4.0,18.0,0.24,2.17,2.42,0.64
2,"Knapp, Jake",T4,-13.0,-5.0,18.0,0.37,1.80,2.16,0.71
3,"Lowry, Shane",T4,-13.0,0.0,18.0,0.83,1.34,2.16,0.81
12,"Conners, Corey",T41,-7.0,-5.0,18.0,-0.61,1.27,0.66,0.76
29,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00
36,0,0,0.0,0.0,0.0,0.00,0.00,0.00,0.00


In [ ]:
# live look at all 48 active players with player as index

print(f"Philly919 has {live_merged[live_merged.team=='Philly919']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Philly919'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"Team Gamble has {live_merged[live_merged.team=='Team Gamble']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='Team Gamble'].drop(columns=['holes_remaining','team'])

In [ ]:
# live look at all 48 active players with player as index

print(f"New Team 4 has {live_merged[live_merged.team=='New Team 4']['holes_remaining'].sum()} Holes Remaining")
live_merged[live_merged.team=='New Team 4'].drop(columns=['holes_remaining','team'])

In [158]:
unit_circle = teams.loc[(teams.team=='unit_circle') & (teams.active_reserve=='Active')].index

live['thru_hp'] = live['thru'] + 18

# live[live.player.isin([unit_circle])]#[['player','total','round','thru','thru_hp']].reset_index(drop=True)

Index(['Patrick Cantlay', 'Tommy Fleetwood', 'Min Woo Lee', 'Corey Conners',
       'Shane Lowry', 'Jake Knapp'],
      dtype='object', name='player')

In [163]:
active_rosters = []

for team in teams.team.unique():
    one_roster = teams.loc[(teams.team==team) & (teams.active_reserve=='Active')].index
    active_rosters.append(one_roster)

cols = teams.team.unique()
active_rosters = pd.DataFrame(active_rosters, index=cols).T
active_rosters

,AlphaWired,unit_circle,Team Gamble,txmoonshine,Philly919,Sneads Foot,Putt Pirates,New Team 4
0,Sam Burns,Patrick Cantlay,Eric Cole,Tom Kim,Luke List,Rory McIlroy,Scottie Scheffler,Cameron Young
1,Xander Schauffele,Tommy Fleetwood,Byeong Hun An,Collin Morikawa,Sungjae Im,Rickie Fowler,Sahith Theegala,Erik Van Rooyen
2,Jordan Spieth,Min Woo Lee,Stephan Jaeger,J.T. Poston,Russell Henley,Sam Ryder,Will Zalatoris,Akshay Bhatia
3,Jason Day,Corey Conners,Matthieu Pavon,Matthew Fitzpatrick,Chris Kirk,Cameron Champ,Emiliano Grillo,Justin Suh
4,Brian Harman,Shane Lowry,Tom Hoge,Keegan Bradley,Justin Rose,Nick Dunlap,Christiaan Bezuidenhout,Camilo Villegas
5,Harris English,Jake Knapp,Chesson Hadley,Nicolai Hojgaard,Daniel Berger,Parker Coody,Kurt Kitayama,Thorbjorn Olesen
